In [1]:
from IPython.display import clear_output
clear_output()

In [2]:
import pathlib
import random
import copy
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn


import seaborn as sns
import matplotlib.pyplot as plt

# Enable GPU. 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import os
os.chdir( "/home/guido/github/spectralgradients" )
print( "Current working directory: ", os.getcwd() )


Current working directory:  /home/guido/github/spectralgradients


In [3]:
# load dataset and model

from collections import OrderedDict
from src.synt_data import SyntDataset, CLASS_DESC

data = SyntDataset(CLASS_DESC)

from torch.utils.data import DataLoader

batch_size = 32

loader = DataLoader(data, batch_size=128, num_workers=8)

x_batch, y_batch = next(iter(loader))


from src.train.model import TimeModule

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
fn = TimeModule.load_from_checkpoint("output/model/checkpoint/epoch=19-step=21686-val_acc=0.93.ckpt", n_classes = data.n_class).eval()
fn = nn.Sequential(OrderedDict([("fn", fn), ("softmax", nn.Softmax(dim=-1))])).to(device)

In [ ]:
from src.explainer_wrapper import explainer_wrapper, localization, lle, irof
from loguru import logger 
from tqdm.autonotebook import tqdm
method = {
    "method": "Saliency",
}

method["model"] = fn

method["irof"] = []
method["lle"] = []

for i in tqdm(torch.arange(0, len(data), batch_size).long()):

    if i + batch_size > len(data):
        batch_size = len(data) - 1

    x_batch, m_batch, y_batch = [], [], []

    for j in range(i, i + batch_size):

        x, m, y = data.__getitem__(j, True)

        # last class is for the baseline 
        if y == data.n_class - 1:
            continue

        x_batch += [x]
        m_batch += [m]
        y_batch += [y]


    x_batch = torch.stack(x_batch)
    m_batch = torch.stack(m_batch)
    y_batch = torch.stack(y_batch)

    method["model"] = fn

    method["inputs"] = x_batch
    method["targets"] = y_batch

    explainer, a_batch = explainer_wrapper(**method)



    logger.info( f"Evaluating {method["method"]} with IROF")
    method["irof"] += [ irof(
        explainer=explainer,
        model=fn,
        x=x_batch,
        y=y_batch,
        attr=a_batch,
        mask=m_batch,
    ) ]

    break


    logger.info( f"Evaluating {method["method"]} with LLE")
    method["lle"] += [ lle(
        explainer= explainer,
        model=fn,
        x=x_batch,
        y=y_batch,
        attr=a_batch,
        mask=m_batch,
    ).item() ]

method["irof"]

  0%|          | 0/1563 [00:00<?, ?it/s]


[tensor(0.0976)]